# 5. scATAC profiles of fetal tissues from the fetal cell atlas dataset
Domcke S, Hill AJ, Daza RM, Cao J, O’Day DR, Pliner HA, et al. A human cell atlas of fetal chromatin accessibility. Science. 2020;370:eaba7612

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Scanpro functions
from scanpro import scanpro
from scanpro.utils import convert_counts_to_df

#Setup path to R for propeller
R_home = os.path.dirname(sys.executable)[:-4] + "/lib/R"
os.environ["R_HOME"] = R_home
%load_ext rpy2.ipython

In [2]:
# path where figures are going to be saved
FIG_PATH = 'plots'

In [3]:
def convert_counts_to_df(counts, prop_cols=None, meta_cols=None, n_cells=1, column_name="Cluster"):
    """ Convert a cell count matrix to a dataframe in long format."""

    counts = counts.copy()

    #If not given, try to get prop_cols and meta_cols automatically
    if prop_cols is None:
        dtypes = counts.dtypes.astype(str)
        prop_cols = [col for i, col in enumerate(counts.columns) if "float" in dtypes[i] or "int" in dtypes[i]]
        
    if meta_cols is None:
        meta_cols = [col for col in counts.columns if col not in prop_cols]

    # Multiply proportions with n_cells
    counts[prop_cols] *= n_cells
    counts[prop_cols] = counts[prop_cols].astype(int)
    
    # Melt into long format (similar to adata.obs)
    counts_melt = pd.melt(counts, id_vars=meta_cols, value_vars=prop_cols, 
                          var_name=column_name, value_name="count")

    # Duplicate rows based on number of cells
    counts_long = counts_melt.loc[counts_melt.index.repeat(counts_melt["count"])].reset_index(drop=True)
    counts_long.drop(columns="count", inplace=True)
    counts_long.index = ["cell_" + str(i) for i in range(1, len(counts_long) + 1)]
    
    return counts_long

------------------

## Read data

In [4]:
fetal_scatac_counts = pd.read_csv("data/scATAC_counts.tsv", sep='\t')

In [5]:
fetal_scatac = convert_counts_to_df(fetal_scatac_counts, column_name="cell_type")

In [6]:
fetal_scatac.head()

,Sample,Sex,Tissue,Batch,Donor_id,cell_type
cell_1,sample_61_pancreas,Female,Pancreas,batch_3,H27917,Acinar cells
cell_2,sample_61_pancreas,Female,Pancreas,batch_3,H27917,Acinar cells
cell_3,sample_61_pancreas,Female,Pancreas,batch_3,H27917,Acinar cells
cell_4,sample_61_pancreas,Female,Pancreas,batch_3,H27917,Acinar cells
cell_5,sample_61_pancreas,Female,Pancreas,batch_3,H27917,Acinar cells


------------

## Run Scanpro

### With replicates

In [7]:
scanpro_out = scanpro.scanpro(fetal_scatac, clusters_col='cell_type',
                                   conds_col='Tissue', samples_col='Sample')

[INFO] The following conditions don't have replicates:  Pancreas, Cerebellum, Spleen
[INFO] Both normal scanpro and sim_scanpro will be performed.
[INFO] Running scanpro with original replicates...
[INFO] There are more than 2 conditions. ANOVA will be performed...


ValueError: f(a) and f(b) must have different signs

In [ ]:
scanpro_out.results

In [ ]:
scanpro_out.plot(n_columns=4, save=f"{FIG_PATH}/scatac_rep_all.pdf")

In [ ]:
scanpro_out.results.to_csv("results/scatac_scanpro_rep.tsv", sep="\t")

### Without replicates

In [ ]:
scanpro_out_norep = scanpro.scanpro(fetal_scatac, clusters_col='cell_type',
                                    conds_col='Tissue')
scanpro_out_norep.results

In [ ]:
scanpro_out_norep.plot(n_columns=4, save=f"{FIG_PATH}/scatac_norep_all.pdf")

In [ ]:
scanpro_out_norep.results.to_csv("results/scatac_scanpro_norep.tsv", sep="\t")

------------------------

## Specific comparisons

### Heart vs. muscle

#### With replicates

In [ ]:
scanpro_out = scanpro.scanpro(fetal_scatac, clusters_col='cell_type',
                              conds_col='Tissue', samples_col='Sample',
                              conditions=["Heart", "Muscle"])
scanpro_out.results.to_csv("results/scatac_scanpro_rep_logit_heart-muscle.tsv", sep="\t")

In [ ]:
scanpro_out = scanpro.scanpro(fetal_scatac, clusters_col='cell_type',
                              conds_col='Tissue', samples_col='Sample',
                              conditions=["Heart", "Muscle"], transform="arcsin")
scanpro_out.results.to_csv("results/scatac_scanpro_rep_arcsin_heart-muscle.tsv", sep="\t")
scanpro_out.results

In [ ]:
scanpro_out.plot(clusters=["Cardiomyocytes", "Myeloid cells", "Smooth muscle cells", "Vascular endothelial cells"], 
                 n_columns=4, save=f"{FIG_PATH}/scatac_rep_arcsin_heart-muscle.pdf")

#### Without replicates

In [ ]:
scanpro_out_norep = scanpro.scanpro(fetal_scatac, clusters_col='cell_type', conds_col='Tissue',
                                    conditions=["Heart", "Muscle"], transform="logit")
scanpro_out_norep.results.to_csv("results/scatac_scanpro_norep_logit_heart-muscle.tsv", sep="\t")

In [ ]:
scanpro_out_norep = scanpro.scanpro(fetal_scatac, clusters_col='cell_type', conds_col='Tissue',
                                    conditions=["Heart", "Muscle"], transform="arcsin")
scanpro_out_norep.results.to_csv("results/scatac_scanpro_norep_arcsin_heart-muscle.tsv", sep="\t")
scanpro_out_norep.results

In [ ]:
scanpro_out_norep.plot(clusters=["Cardiomyocytes", "Myeloid cells", "Smooth muscle cells", "Vascular endothelial cells"], 
                       n_columns=4, save=f"{FIG_PATH}/scatac_norep_arcsin_heart-muscle.pdf")

### Heart vs. intestine vs. muscle

#### With replicates

In [ ]:
scanpro_out = scanpro.scanpro(fetal_scatac, clusters_col='cell_type',
                              conds_col='Tissue', samples_col='Sample',
                              conditions=["Heart", "Intestine", "Muscle"])
scanpro_out.results.to_csv("results/scatac_scanpro_rep_logit_heart-intestine-muscle.tsv", sep="\t")

In [ ]:
scanpro_out = scanpro.scanpro(fetal_scatac, clusters_col='cell_type',
                              conds_col='Tissue', samples_col='Sample',
                              conditions=["Heart", "Intestine", "Muscle"], transform="arcsin")
scanpro_out.results.to_csv("results/scatac_scanpro_rep_arcsin_heart-intestine-muscle.tsv", sep="\t")
scanpro_out.results

In [ ]:
scanpro_out.plot(clusters=["Cardiomyocytes", "Myeloid cells", "Smooth muscle cells", "Vascular endothelial cells"], n_columns=4,
                 save=f"{FIG_PATH}/scatac_rep_arcsin_heart-intestine-muscle.pdf")

#### No replicates

In [ ]:
scanpro_out_norep = scanpro.scanpro(fetal_scatac, clusters_col='cell_type',
                              conds_col='Tissue',
                              conditions=["Heart", "Intestine", "Muscle"])
scanpro_out_norep.results.to_csv("results/scatac_scanpro_norep_logit_heart-intestine-muscle.tsv", sep="\t")

In [ ]:
scanpro_out_norep = scanpro.scanpro(fetal_scatac, clusters_col='cell_type',
                              conds_col='Tissue',
                              conditions=["Heart", "Intestine", "Muscle"], transform="arcsin")
scanpro_out_norep.results.to_csv("results/scatac_scanpro_norep_arcsin_heart-intestine-muscle.tsv", sep="\t")

In [ ]:
scanpro_out_norep.plot(clusters=["Cardiomyocytes", "Myeloid cells", "Smooth muscle cells", "Vascular endothelial cells"], 
                       n_columns=4, save=f"{FIG_PATH}/scatac_norep_arcsin_heart-intestine-muscle.pdf")

-----------

## Propeller

In [ ]:
fetal_scatac_subset = fetal_scatac[fetal_scatac["Tissue"].isin(["Heart", "Muscle"])]

In [ ]:
%%R -i fetal_scatac_subset -o propeller_results_logit -o propeller_results_arcsin
library(speckle)

fetal_scatac = fetal_scatac_subset

propeller_results_logit = propeller(clusters = fetal_scatac$cell_type, sample = fetal_scatac$Sample, group = fetal_scatac$Tissue, transform="logit")
propeller_results_arcsin = propeller(clusters = fetal_scatac$cell_type, sample = fetal_scatac$Sample, group = fetal_scatac$Tissue, transform="arcsin")

In [ ]:
propeller_results

In [ ]:
propeller_results_logit.to_csv("results/scatac_propeller_rep_logit.tsv", sep="\t")
propeller_results_arcsin.to_csv("results/scatac_propeller_rep_arcsin.tsv", sep="\t")